<a href="https://colab.research.google.com/github/quanvu0996/compet/blob/master/kalapa_4student_net2_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [153]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, DenseFeatures
from tensorflow.keras import Sequential
from tensorflow.keras.metrics import AUC, Precision, Recall
import tensorflow.keras.backend as K
from tensorflow import feature_column as fc
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

%matplotlib inline

In [154]:
print(pd.__version__)
print(tf.__version__)

1.0.5
2.3.0


## Load data

In [155]:
submision_path = '/content/drive/My Drive/Data/colabs_data/kalapa_4students/simple_submission.csv'
train_path = '/content/drive/My Drive/Data/colabs_data/kalapa_4students/train.csv'
test_path = '/content/drive/My Drive/Data/colabs_data/kalapa_4students/test.csv'

In [156]:
def load_data(train_path, test_path, label_col = 'label'):
    train_set = pd.read_csv(train_path)
    test_set = pd.read_csv(test_path)
    return train_set, test_set

train_set, test_set = load_data( train_path, test_path )
cols = train_set.columns

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (35,43) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (34,42) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [157]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53030 entries, 0 to 53029
Columns: 195 entries, id to Field_82
dtypes: float64(133), int64(3), object(59)
memory usage: 78.9+ MB


### Data trasforming

In [158]:
# train_set[cols_categorical].head()
# train_set.Field_61.unique()

In [159]:
# len(train_set.Field_45.unique())
# cols_id

In [160]:
# len(train_set.homeTownCity.unique())

In [161]:
# train_set.Field_75.unique()
# train_set.homeTownCity.hist()
# print(cols_docs)

In [162]:
train_set[cols_fts_num].head()

,Field_3,Field_19,Field_20,Field_21,Field_22,Field_23,Field_27,Field_28,Field_29,namSinh,Field_58,Field_59,Field_60,Field_63,Field_64,Field_67,Field_69,Field_70,Field_71,Field_72,Field_74,Field_75,Field_76,Field_77,friendCount,subscriberCount,currentLocationLocationId,currentLocationLatitude,currentLocationLongitude,homeTownLocationId,homeTownLatitude,homeTownLongitude,topFriends,numOrg,F_numOrg,F_numQuery,E_numOrg,E_numQuery,C_numOrg,C_numQuery,G_numOrg,G_numQuery,A_numOrg,A_numQuery,summary_6m,summary_3m,summary_1m,summary_1w,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82
0,1.0,0.0,4258600.0,4.5,1.0,16.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,2.0,1.0,118410.0,0.00,0.49,0.245,0.49,NaN,5.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,704.0,6.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
2,2.0,0.0,5000000.0,4.5,1.0,10.0,0.0,0.0,0.0,0.0,0.0,5.0,3.0,3.0,2.0,1073365.0,0.58,0.77,0.675,0.19,1076.0,2.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,1.0,1.0,2.0,3.0,NaN,NaN,NaN,NaN,3.0,3.0,7.0,7.0,2.0,1.0,1.0,33.0,10.769445,6.466667,0.0,2
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,33.0,10.769445,6.466667,0.0,3
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,33.0,10.769445,6.466667,0.0,3


In [163]:
# Nhãn
cols_label = ['label']
# Đặc trưng binary
cols_fts_binary = [i for i in train_set.select_dtypes(include=['float64','int64']).columns 
                   if len(set(train_set[i].fillna(0)) - {0,1} ) == 0 and i not in cols_label]
# định dang datte yyyy-mm-dd
cols_date = ['Field_1','Field_2','Field_5','Field_6','Field_7','Field_8','Field_9','Field_10','Field_11'
             ,'Field_14','Field_15','Field_25','Field_32','Field_33','Field_34','Field_35','Field_40','Field_43','Field_44'
             ,'F_startDate','F_endDate','E_startDate','E_endDate','C_startDate','C_endDate','G_startDate','G_endDate'
             ,'A_startDate','A_endDate']
# định dạng date yyymmdd
cols_date2 = ['Field_33','ngaySinh']
# Đặc trưng dạng văn bản
cols_docs = ['Field_46','diaChi','Field_48','Field_49','currentLocationName','homeTownName','Field_56']
# Định danh bản ghi
cols_id = ['id','Field_45'] +[i for i in train_set.select_dtypes(include = ['object']).columns 
                              if len(train_set[i].unique()) >=350 and i not in cols_date + cols_date2+cols_docs]
# Đặc trưng dạng categorical
cols_categorical = [i for i in train_set.select_dtypes(include = ['object']).columns
                    if i not in cols_id + cols_label+cols_date2+cols_date+cols_docs]
# Đặc trưng số
cols_fts_num = [ i for i in test_set.select_dtypes(include=['float64','int64']).columns 
                if i not in cols_id + cols_label + cols_fts_binary + cols_date2+cols_date+cols_docs]

In [164]:
# Đặc trưng số có dạng như categorical
cols_num_like_cat = [i for i in cols_fts_num if len(train_set[i].unique()) <= 15 ]
# Đặc trưng số đã kiểm chứng
cols_num = [i for i in cols_fts_num if i not in cols_num_like_cat]

In [165]:
for i in cols_fts_num :
  print(i, len(train_set[i].unique()))

Field_3 3
Field_19 355
Field_20 4736
Field_21 5
Field_22 4
Field_23 314
Field_27 637
Field_28 635
Field_29 44
namSinh 4
Field_58 6
Field_59 36
Field_60 35
Field_63 11
Field_64 10
Field_67 14431
Field_69 102
Field_70 51
Field_71 322
Field_72 497
Field_74 490
Field_75 15
Field_76 12
Field_77 16
friendCount 2933
subscriberCount 1526
currentLocationLocationId 915
currentLocationLatitude 680
currentLocationLongitude 590
homeTownLocationId 932
homeTownLatitude 643
homeTownLongitude 522
topFriends 6
numOrg 20
F_numOrg 7
F_numQuery 16
E_numOrg 8
E_numQuery 16
C_numOrg 4
C_numQuery 77
G_numOrg 5
G_numQuery 11
A_numOrg 15
A_numQuery 37
summary_6m 104
summary_3m 95
summary_1m 81
summary_1w 52
num_of_phone 6
Field_78 428
Field_79 11037
Field_80 2840
Field_81 422
Field_82 4


Missing values

In [166]:
train_data = train_set
predict_data = test_set

# Categorical: Fill giá trị null như một giá trị khác
train_data[cols_categorical] = train_set[cols_categorical].astype('object').fillna('#NULL#').applymap(str)
predict_data[cols_categorical] = test_set[cols_categorical].astype('object').fillna('#NULL#').applymap(str)

# Numerical: các trường chưa xử lý+ numerical fill giá trị -1
# train_data[cols_fts_num] = train_set[cols_fts_num].fillna(-1)
# predict_data[cols_fts_num] = test_set[cols_fts_num].fillna(-1)
X_train[cols_fts_num] = (X_train[cols_fts_num] - X_train[cols_fts_num].mean() )/ X_train[cols_fts_num].std()
X_test[cols_fts_num] = (X_test[cols_fts_num] - X_train[cols_fts_num].mean() )/ X_train[cols_fts_num].std()
train_data = train_set.fillna(-1)
predict_data = test_set.fillna(-1)

Định nghĩa features

In [167]:
X_train, X_test, y_train, y_test = train_test_split(train_data
                                                   ,train_data['label'].values
                                                   , stratify = train_data['label'].values 
                                                   ,test_size = 0.3 
                                                   )

In [168]:
feature_columns = []
fts_cols = cols_fts_binary+cols_fts_num+cols_categorical

# Với biến binary, load trực tiếp vào model
for col in cols_fts_binary:
  feature_columns.append(
      fc.numeric_column(col, dtype=tf.float32)
  )
# Với các biến numeric, sử dụng standard scaler + định nghĩa
# X_train[cols_fts_num] = (X_train[cols_fts_num] - X_train[cols_fts_num].mean() )/ X_train[cols_fts_num].std()
# X_test[cols_fts_num] = (X_test[cols_fts_num] - X_train[cols_fts_num].mean() )/ X_train[cols_fts_num].std()

for col in cols_fts_num:
  mean = X_train[col].mean()
  std = X_train[col].std()
  feature_columns.append(
      fc.numeric_column(col,
                        dtype=tf.float32
                        #, normalizer_fn = lambda x: ( x - mean )/ std 
                        ))
# Với các biến categorical, embed với số chiều bằng 1/200 +1 số chiều gốc
def embed_trans(feature_name, vocab):
  ft = fc.categorical_column_with_vocabulary_list(feature_name, vocab)
  ft = fc.embedding_column( ft,dimension= int(len(vocab)/20) +1 )
  return ft
for col in cols_categorical:
  vocabulary = train_set[col].unique()
  feature_columns.append(
      embed_trans(col, vocabulary)
  )


# Xây dựng layer
feature_layer = DenseFeatures(feature_columns)

In [169]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(X, y, shuffle=True, batch_size=32):
  ds = tf.data.Dataset.from_tensor_slices((X.to_dict(orient='list'), y))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(X))
  ds = ds.batch(batch_size)
  return ds


batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(X_train[fts_cols], y_train, batch_size=batch_size)
test_ds = df_to_dataset(X_test[fts_cols], y_test, shuffle=False, batch_size=batch_size)

In [170]:
# scaler_1 = StandardScaler()
# X_train_scaled = scaler_1.fit_transform(X_train)
# X_test_scaled = scaler_1.transform(X_test)
# X_predict_scaled = scaler_1.transform( test_set[cols_fts_num].fillna(0) )

In [171]:
nets_2 = Sequential(
    [
    feature_layer,
    Dense(128, activation ='relu'),
    # Dense(64, activation ='relu'),
    # Dense(64, activation ='relu'),
    Dense(64, activation ='relu'),
    Dense(16, activation ='relu'),
    Dense(1, activation ='sigmoid')])
nets_2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc', AUC() ] )#, Precision(), Recall()] )

In [172]:
history = nets_2.fit( train_ds , epochs = 10
                    , validation_data = test_ds)

Epoch 1/10
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
3742/7425 [==============>...............] - ETA: 26s - loss: 32572850176.0000 - acc: 0.6086 - auc_12: 0.5373

KeyboardInterrupt: ignored

Dự đoán và xuất file kế quả

In [ ]:
y_predict_pred= nets_2.predict(X_predict_scaled)

In [ ]:
res_df = pd.DataFrame({'id': test_set.id, 'label': y_predict_pred.reshape(y_predict_pred.shape[0])})
res_df.to_csv('simple_submission.csv', index=False)